In [1]:
import pandas as pd
import time
import os
import sys
import zarr
import numpy as np
import napari
import dask.array as da

In [2]:
df = pd.read_pickle('/Users/apple/Desktop/CME-ANALYSIS/Image-Analysis-Tracking/Final/test_data/datasets/cleaned_spots_intensities_c3_all.pkl')

In [3]:
df.head()

,amplitude,mu_x,mu_y,mu_z,sigma_x,sigma_y,sigma_z,frame
0,206.593249,11.0,12.0,0.0,2.0,2.0,4.0,0
1,191.000000,36.0,88.0,1.0,2.0,1.0,2.0,0
2,231.007679,40.0,141.0,1.0,2.0,2.0,2.0,0
3,185.152538,40.0,58.0,1.0,2.0,1.0,3.0,0
4,202.000000,46.0,73.0,1.0,2.0,2.0,3.0,0


In [4]:
# This assumes that your notebook is inside 'Jupyter Notebooks', which is at the same level as 'test_data'
base_dir = os.path.join(os.path.dirname(os.path.abspath("__file__")), '..', 'test_data')

zarr_directory = 'zarr_file/all_channels_data'
zarr_full_path = os.path.join(base_dir, zarr_directory)

z2 = zarr.open(zarr_full_path, mode='r')

In [5]:
z2.shape

(130, 3, 75, 258, 275)

In [6]:
def create_mask(frame_number, zarr_obj, detections_dataframe):
    
    total_frames = zarr_obj.shape[0]
    z_max = zarr_obj.shape[2]
    y_max = zarr_obj.shape[3]
    x_max = zarr_obj.shape[4]

    
    if frame_number >= total_frames: 
        raise valueError("frame number provided is out of bounds of the frames in the movie")
    else: 
        mask_array = np.zeros((1,z_max,y_max, x_max))

    current_time_df = detections_dataframe[detections_dataframe['frame'] == frame_number].copy(deep = True)
    
    #dropping spots out of bounds of z axis 
    condition_1 = current_time_df['mu_z'] >= 0
    condition_2 = current_time_df['mu_z'] <= z2.shape[2]
    
    #dropping spots out of bounds of x axis 
    condition_3 = current_time_df['mu_x'] >= 0
    condition_4 = current_time_df['mu_x'] <= z2.shape[4]
    
    #dropping spots out of bounds of y axis 
    condition_5 = current_time_df['mu_y'] >= 0
    condition_6 = current_time_df['mu_y'] <= z2.shape[3]

    current_time_df = current_time_df[condition_1 & condition_2 & condition_3 & condition_4 & 
    condition_5 & condition_6].reset_index(drop = True)

    for row in df.itertuples(index=True, name='Pandas'):
        z = row[4]
        y = row[3]
        x = row[2]
        sigma_z = row[7]

        sigma_y = row[6]

        sigma_x = row[5]


        z_lower_bound = int(max(0,z - sigma_z))
        z_upper_bound = int(min(z_max,z + sigma_z))

        y_lower_bound = int(max(0, y - sigma_y))
        y_upper_bound = int(min(y_max, y + sigma_y))

        x_lower_bound = int(max(0, x - sigma_x))
        x_upper_bound = int(min(x_max, x + sigma_x))

        mask_array[0,z_lower_bound:z_upper_bound,y_lower_bound:y_upper_bound,x_lower_bound:x_upper_bound] = 1   

    return mask_array
        

In [7]:
detections_masks = create_mask(frame_number = 0, zarr_obj = z2, detections_dataframe = df)

In [8]:
# Check if there are any zeros in the array
contains_zeros = np.any(detections_masks == 0)

# Print the result
if contains_zeros:
    print("The array contains zeros.")
else:
    print("The array does not contain any zeros.")

The array contains zeros.


In [9]:
# Create a napari viewer
viewer = napari.Viewer()

#access channel 3 only from zarr array 
dask_array = da.from_zarr(z2)

#the axis arrangement is (t,c,z,y,x)
#for the sake of improved performance only 1 channel could be imported here (if images get super large and performance issues occur)
all_channels = dask_array[:,:,:,:,:]

# Add the 4D stack to the viewer
layer_raw = viewer.add_image(all_channels, channel_axis = 1, name = ['actin', 'dynamin', 'clathrin'])
#other useful parameters 
#color_map = list
#contrast_limits = list of list 

# Add Bounding Box
layer_raw[0].bounding_box.visible = True
layer_raw[1].bounding_box.visible = True
layer_raw[2].bounding_box.visible = True

In [10]:
#viewer.add_image(detections_masks, name = 'masks')


In [14]:
import numpy as np

def visualize_3D_gaussians(zarr_obj, gaussians_df):
    '''
    This function takes a raw image from which gaussians have been segmented
    and fitted, and a DataFrame containing the Gaussian parameters to draw them
    into a new 3D array that can then be visualized or outputted as a TIFF.
    '''

    
    image_gaussians = np.zeros((zarr_obj.shape[2],zarr_obj.shape[3],zarr_obj.shape[4]))

    # Extract Gaussian parameters from the DataFrame
    amplitudes = gaussians_df['amplitude'].values * 100
    mu_xs = gaussians_df['mu_x'].values.astype(int)
    mu_ys = gaussians_df['mu_y'].values.astype(int)
    mu_zs = gaussians_df['mu_z'].values.astype(int)
    sigma_xs = gaussians_df['sigma_x'].values
    sigma_ys = gaussians_df['sigma_y'].values
    sigma_zs = gaussians_df['sigma_z'].values

    for amplitude, mu_x, mu_y, mu_z, sigma_x, sigma_y, sigma_z in zip(amplitudes, mu_xs, mu_ys, mu_zs, sigma_xs, sigma_ys, sigma_zs):
        n_neighbors_x = int(3 * sigma_x) + 1
        n_neighbors_y = int(3 * sigma_y) + 1
        n_neighbors_z = int(3 * sigma_z) + 1

        z_range = np.arange(max(0, mu_z - n_neighbors_z), min(image_gaussians.shape[0], mu_z + n_neighbors_z + 1))
        y_range = np.arange(max(0, mu_y - n_neighbors_y), min(image_gaussians.shape[1], mu_y + n_neighbors_y + 1))
        x_range = np.arange(max(0, mu_x - n_neighbors_x), min(image_gaussians.shape[2], mu_x + n_neighbors_x + 1))

        zz, yy, xx = np.meshgrid(z_range, y_range, x_range, indexing='ij')
        distances = (
            ((zz - mu_z) ** 2) / (2 * sigma_z ** 2) +
            ((yy - mu_y) ** 2) / (2 * sigma_y ** 2) +
            ((xx - mu_x) ** 2) / (2 * sigma_x ** 2)
        )
        gaussian_values = amplitude * np.exp(-distances)
        np.add.at(image_gaussians, (zz, yy, xx), gaussian_values)
    
    return image_gaussians


In [15]:
temp = visualize_3D_gaussians(zarr_obj = z2, gaussians_df = df)

In [16]:
viewer.add_image(temp, name = 'masks')


<Image layer 'masks [1]' at 0x7fab0e2410c0>